In [ ]:
import numpy as np
from parcels import FieldSet, Field
from scipy.io import savemat
import xarray as xr

### vector field

In [ ]:
lat_size = 25
lon_size = 25
u = np.full((lat_size, lon_size), np.nan)
v = np.full((lat_size, lon_size), np.nan)
lat_diff = 0.00898933
lon_diff = 0.01042175
u_val = 0.05
v_val = -0.05

lon_right = -117.12
lat_bottom = 32.35

lats = np.array([lat_bottom + i * lat_diff for i in range(lat_size)])
lons = np.array([lon_right - i * lon_diff for i in reversed(range(lon_size))])

start_time = np.datetime64("2020-02-09T00")
end_time = np.datetime64("2020-02-14T00")
times = np.arange(start_time, end_time)

### coastline data

In [ ]:
import random

# vertical coastline
coast_lon = -117.16
coast_lats = [lat_bottom]
while coast_lats[-1] <= lats[-1]:
    coast_lats.append(coast_lats[-1] + lat_diff * random.uniform(0.5, 1.5))
coast_lats = np.array(coast_lats)
coast_lons = np.array([coast_lon for _ in coast_lats])

In [ ]:
mgrid = np.meshgrid(lats, lons, indexing="ij")
# u[mgrid[1] < coast_lon] = u_val
u[:] = u_val
# v[mgrid[1] < coast_lon] = v_val
v[:] = v_val

### generate dataset and nc file

In [ ]:
u_all = np.tile(u, (len(times), 1, 1))
v_all = np.tile(v, (len(times), 1, 1))
ds = xr.Dataset(
    data_vars={
        "U": (["time", "lat", "lon"], u_all),
        "V": (["time", "lat", "lon"], v_all),
    },
    coords={"time": times, "lat": lats, "lon": lons},
)
ds.to_netcdf("data/field_netcdfs/dummy_field.nc")

### save coastline into mat file

In [ ]:
dummy_mat_dict = {"rlatz": coast_lats, "rlonz": coast_lons}
savemat("matlab/dummy_coast.mat", dummy_mat_dict)

### spawn points lol

In [ ]:
sp_lats = np.linspace(lats[-1] - lat_diff * 10, lats[-1], 5)
sp_lons = np.linspace(lons[-1] - lon_diff * 15, lons[-1] - lon_diff * 5, 5)

In [ ]:
sp_mgrid = np.meshgrid(sp_lats, sp_lons, indexing="ij")
sp_all_lats = sp_mgrid[0].flatten()
sp_all_lons = sp_mgrid[1].flatten()
dummy_sp_dict = {"xf": sp_all_lons, "yf": sp_all_lats}
savemat("matlab/dummy_spawns.mat", dummy_sp_dict)